## EnumOutputParser

- LangChain의 EnumOutputParser는 언어 모델의 출력을 미리 정의된 열거형(Enum) 값 중 하나로 파싱하는 도구.
- 이 파서의 주요 특징과 사용법은 다음과 같다
  - 주요 특징
    - **열거형 파싱**: 문자열 출력을 미리 정의된 Enum 값으로 변환
    - **타입 안전성**: 파싱된 결과가 반드시 정의된 Enum 값 중 하나임을 보장
    - **유연성**: 공백이나 줄바꿈 문자를 자동으로 처리
  - 사용 방법
    - EnumOutputParser는 언어 모델의 출력에서 유효한 Enum 값을 추출하는 데 유용
    - 이를 통해 출력 데이터의 일관성을 유지하고 예측 가능성을 높일 수 있음
    - 파서를 사용하려면, 미리 정의된 Enum 값을 설정하고 해당 값을 기준으로 문자열 출력을 파싱

In [2]:
from enum import Enum
from langchain.output_parsers.enum import EnumOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_teddynote import logging
from dotenv import load_dotenv

load_dotenv()
logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


- `enum` 모듈을 사용하여 `Colors` 클래스를 정의
- `Colors` 클래스는 `Enum`을 상속받으며, `RED`, `GREEN`, `BLUE` 세 가지 색상 값

In [3]:
from enum import Enum

class Colors(Enum):
    RED = "빨간색"
    GREEN = "초록색"
    BLUE = "파란색"

In [4]:
Colors.RED

<Colors.RED: '빨간색'>

In [5]:
parser = EnumOutputParser(enum=Colors)
parser.get_format_instructions()
print(parser.get_format_instructions())

Select one of the following options: 빨간색, 초록색, 파란색


In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template(
    """다음의 물체는 어떤 색깔인가요?

Object: {object}

Instructions: {instructions}"""
    # 파서에서 지시사항 형식을 가져와 부분적으로 적용합니다.
).partial(instructions=parser.get_format_instructions())

# 체인을 생성합니다.
chain = prompt | ChatOpenAI() | parser

- "하늘"에 대한 정보 요청

In [7]:
response = chain.invoke({"object": "하늘"})
print(response)

Colors.BLUE


In [10]:
# 출력의 타입을 확인
type(response)

<enum 'Colors'>

In [11]:
# 출력의 값을 확인
response.value

'파란색'